# 因果语言模型训练实例

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, BloomForCausalLM

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")

def process_func(examples):
    contents = [e + tokenizer.eos_token for e in examples["completion"]]
    return tokenizer(contents, max_length=384, truncation=True)

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))

In [8]:
next(enumerate(dl))

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [11]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [12]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

## Step4 创建模型

In [13]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

## Step5 配置训练参数

In [14]:
args = TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [15]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

## Step7 模型训练

In [16]:
trainer.train()

c:\Users\yuyao\miniconda3\envs\transformers\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/312 [00:00<?, ?it/s]

{'loss': 3.985, 'learning_rate': 4.83974358974359e-05, 'epoch': 0.03}
{'loss': 3.9901, 'learning_rate': 4.67948717948718e-05, 'epoch': 0.06}
{'loss': 3.8418, 'learning_rate': 4.519230769230769e-05, 'epoch': 0.1}
{'loss': 3.8249, 'learning_rate': 4.358974358974359e-05, 'epoch': 0.13}
{'loss': 3.6815, 'learning_rate': 4.198717948717949e-05, 'epoch': 0.16}
{'loss': 3.6652, 'learning_rate': 4.038461538461539e-05, 'epoch': 0.19}
{'loss': 3.6319, 'learning_rate': 3.878205128205129e-05, 'epoch': 0.22}
{'loss': 3.6918, 'learning_rate': 3.717948717948718e-05, 'epoch': 0.26}
{'loss': 3.6513, 'learning_rate': 3.557692307692308e-05, 'epoch': 0.29}
{'loss': 3.6396, 'learning_rate': 3.397435897435898e-05, 'epoch': 0.32}
{'loss': 3.5632, 'learning_rate': 3.2371794871794876e-05, 'epoch': 0.35}
{'loss': 3.5992, 'learning_rate': 3.0769230769230774e-05, 'epoch': 0.38}
{'loss': 3.6086, 'learning_rate': 2.916666666666667e-05, 'epoch': 0.42}
{'loss': 3.5191, 'learning_rate': 2.756410256410257e-05, 'epoch': 

TrainOutput(global_step=312, training_loss=3.58796650935442, metrics={'train_runtime': 374.5245, 'train_samples_per_second': 26.701, 'train_steps_per_second': 0.833, 'train_loss': 3.58796650935442, 'epoch': 1.0})

## Step8 模型推理

In [17]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [23]:
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安", max_length=128, do_sample=True)

[{'generated_text': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学西安学院西楼三期大楼的二层混合式博物馆。该建筑占地约2000平方米，于2012年落成开馆，是西安交通大学博物馆的一个组成部分，由西安交通大学及陕西省西安市设计与艺术设计研究院联合发起筹建。该建筑是西安交通大学学生宿舍（西安理工大学建筑学院宿舍）及学生食堂（西安理工大学建筑学院食堂）的一部份，建成后将极大地方便西安交通大学所有学生的住宿和出行。\n博物馆馆舍\n博物馆位于西安交通大学西安学院西楼3号楼，由陕西西安设计院"}]

In [25]:
pipe("下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常", max_length=128, do_sample=True)

[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常之盛。而随着这几年游戏的高速发展，玩家对于游戏的了解越来越强烈，以至于在游戏行业里，“烧钱”或“低成本”的定义也逐渐淡出了人们的视线。\n虽然，这是指游戏产业，但这仅仅是以游戏厂商，而不是游戏开发商为准。\n娱乐产业\n近期，随着游戏产业的发展，大量有潜力的电子游戏公司，如电子游戏工作室Game Factory已经或正在开发大型商业化游戏，并且已经确定了。这些游戏产业的公司，有的是以研发游戏，而有一部分，如N'}]